<a href="https://colab.research.google.com/github/cisimon7/Kinematics-of-Redundant-Robot/blob/main/Redundant_Robots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Work done by:

### <b><i>Cham An Fam</i></b>
### <b><i>Simon Idoko</i></b>
### <b><i>Ruslan Damindarov</i></b>

<h1>KUKA iiwa LBR14 R820</h1>

### <b><i>Imports</i></b>

In [1]:
import time
import numpy as np
import sympy as sym
sym.init_printing()
from math import atan2, sqrt
import matplotlib.pyplot as plt
from IPython.display import clear_output 

In [2]:
np.set_printoptions(precision=4, suppress=True, linewidth=200)

### <b><i>Utilities</i></b>

##### Rotation Matrices

In [3]:
def Rx(q):
  T = np.array([[1,         0,          0, 0],
                [0, np.cos(q), -np.sin(q), 0],
                [0, np.sin(q),  np.cos(q), 0],
                [0,         0,          0, 1]], dtype=float)
  return T

def Ry(q):
  T = np.array([[ np.cos(q), 0, np.sin(q), 0],
                [         0, 1,         0, 0],
                [-np.sin(q), 0, np.cos(q), 0],
                [         0, 0,         0, 1]], dtype=float)
  return T

def Rz(q):
  T = np.array([[np.cos(q), -np.sin(q), 0, 0],
                [np.sin(q),  np.cos(q), 0, 0],
                [        0,          0, 1, 0],
                [        0,          0, 0, 1]], dtype=float)
  return T

In [4]:
def Rx_sym(q):
  return sym.Matrix(
      [[1, 0, 0, 0],
        [0, sym.cos(q), -sym.sin(q), 0],
        [0, sym.sin(q), sym.cos(q), 0],
        [0, 0, 0, 1]]
  )

def Ry_sym(q):
  return sym.Matrix(
      [[sym.cos(q), 0, sym.sin(q), 0],
        [0, 1, 0, 0],
        [-sym.sin(q), 0, sym.cos(q), 0],
        [0, 0, 0, 1]]
  )

def Rz_sym(q):
  return sym.Matrix(
      [[sym.cos(q), -sym.sin(q), 0, 0],
        [sym.sin(q), sym.cos(q), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]
  )

##### Rotation Derivative Matrices

In [5]:
def d_Rx(q):
  T = np.array([[0,          0,          0, 0],
                [0, -np.sin(q), -np.cos(q), 0],
                [0,  np.cos(q), -np.sin(q), 0],
                [0,          0,          0, 0]], dtype=float)
  return T

def d_Ry(q):
  T = np.array([[-np.sin(q), 0,  np.cos(q), 0],
                [         0, 0,          0, 0],
                [-np.cos(q), 0, -np.sin(q), 0],
                [         0, 0,          0, 0]], dtype=float)
  return T

def d_Rz(q):
  T = np.array([[-np.sin(q), -np.cos(q), 0, 0],
                [ np.cos(q), -np.sin(q), 0, 0],
                [         0,          0, 0, 0],
                [         0,          0, 0, 0]], dtype=float)
  return T

In [6]:
def d_Rx_sym(q):
  return sym.Matrix(
      [[0, 0, 0, 0],
        [0, -sym.sin(q), -sym.cos(q), 0],
        [0, sym.cos(q), -sym.sin(q), 0],
        [0, 0, 0, 0]]
  )

def d_Ry_sym(q):
  return sym.Matrix(
      [[-sym.sin(q), 0, sym.cos(q), 0],
        [0, 0, 0, 0],
        [-sym.cos(q), 0, -sym.sin(q), 0],
        [0, 0, 0, 0]]
  )

def d_Rz_sym(q):
  return sym.Matrix(
      [[-sym.sin(q), -sym.cos(q), 0, 0],
        [sym.cos(q), -sym.sin(q), 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]])

##### Translation Matrices

In [7]:
def Tx(x):
  T = np.array([[1, 0, 0, x],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]], dtype=float)
  return T

def Ty(y):
  T = np.array([[1, 0, 0, 0],
                [0, 1, 0, y],
                [0, 0, 1, 0],
                [0, 0, 0, 1]], dtype=float)
  return T

def Tz(z):
  T = np.array([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, z],
                [0, 0, 0, 1]], dtype=float)
  return T

In [8]:
def Tx_sym(s):
  return sym.Matrix(
      [[1, 0, 0, s],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]
  )

def Ty_sym(s):
  return sym.Matrix(
      [[1, 0, 0, 0],
        [0, 1, 0, s],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]
  )

def Tz_sym(s):
  return sym.Matrix(
      [[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, s],
        [0, 0, 0, 1]]
  )

##### Translation Derivative Matrices

In [9]:
def d_Tx(x):
  T = np.array([[0, 0, 0, 1],
                [0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0]], dtype=float)
  return T

def d_Ty(y):
  T = np.array([[0, 0, 0, 0],
                [0, 0, 0, 1],
                [0, 0, 0, 0],
                [0, 0, 0, 0]], dtype=float)
  return T

def d_Tz(z):
  T = np.array([[0, 0, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 1],
                [0, 0, 0, 0]], dtype=float)
  return T

In [10]:
def d_Tx_sym():
  return sym.Matrix(
      [[0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]
  )

def d_Ty_sym():
  return sym.Matrix(
      [[0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]
  )

def d_Tz_sym():
  return sym.Matrix(
      [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0]]
  )

##### Euler Angles from Rotation Matrix

In [11]:
def euler_angles(R, sndSol=True):
  rx = atan2(R[2,0], R[2,1])
  ry = atan2(sqrt(R[0,2]**2 + R[1,2]**2), R[2,2])
  rz = atan2(R[0,2], -R[1,2])

  return [rx, ry, rz]

### <b><i>Parameters And Trajectory Definitions</i></b> 

In [12]:
# Length of Links in meters
a1, a2, a3, a4 = 0.36, 0.42, 0.4, 0.126

pi = np.pi
pi_sym = sym.pi

P = np.sin(np.linspace(-2.5,2.5))

### <b><i>Forward Kinematics</i></b>

In [13]:
def FK(joint_params):
  """
  Joint variables consisting of 7 parameters
  """
  joint_params = np.asarray(joint_params, dtype=float)
  q1, q2, q3, q4, q5, q6, q7 = joint_params
  TF = np.linalg.multi_dot([ 
          Rz(q1), Tz(a1),           # Joint 1 to 2
          Rx(q2),                   # Joint 2 to 3 
          Rz(q3), Tz(a2),           # Joint 3 to 4 
          Rx(q4),                   # Joint 4 to 5  
          Rz(q5), Tz(a3),           # Joint 5 to 6 
          Rx(q6),                   # Joint 6 to 7 
          Rz(q7), Tz(a4)            # Joint 7 to E
  ])

  return TF

### <b><i>Jacobian matrix</i></b>

In [14]:
def jacobian_sym():
  q1, q2, q3, q4, q5, q6, q7 = sym.symbols("q_1 q_2 q_3 q_4 q_5 q_6 q_7", real=True)  

  variables = [q1, q2, q3, q4, q5, q6, q7]

  TF =  Rz_sym(q1) @ Tz_sym(a1) @ \
        Rx_sym(q2)              @ \
        Rz_sym(q3) @ Tz_sym(a2) @ \
        Rx_sym(q4)              @ \
        Rz_sym(q5) @ Tz_sym(a3) @ \
        Rx_sym(q6)              @ \
        Rz_sym(q7) @ Tz_sym(a4)

  R = TF[:3,:-1]
  jacobian = sym.Matrix([])

  for var in variables:
      T_d  = sym.diff(TF, var) 

      T    = T_d[0:3, -1]
      R_d  = T_d[0:3, :-1]
      R_j  = R_d @ R.T 

      J = T.row_insert(3, sym.Matrix([R_j[2,1], R_j[0,2], R_j[1,0]]))

      jacobian = jacobian.col_insert(len(jacobian), J)

  return sym.lambdify([variables], jacobian, "numpy")

jacobian_sym_func = jacobian_sym()

def jacobian(joint_params):
    variables = [*joint_params]
    return jacobian_sym_func(variables)

### <b><i>Plot Function</i></b>

In [15]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [16]:
def plot_robot(q_parms):

  q1, q2, q3, q4, q5, q6, q7 = q_parms

  T01 = np.eye(4)
  T12 = Rz(q1) @ Tz(a1)           # Joint 1 to 2
  T23 = Rx(q2)                    # Joint 2 to 3
  T34 = Rz(q3) @ Tz(a2)           # Joint 3 to 4
  T45 = Rx(q4)                    # Joint 4 to 5
  T56 = Rz(q5) @ Tz(a3)           # Joint 5 to 6
  T67 = Rx(q6)                    # Joint 6 to 7
  T7E = Rz(q7) @ Tz(a4)           # Joint 7 to E
  
  T02 = T01 @ T12
  T03 = T01 @ T12 @ T23
  T04 = T01 @ T12 @ T23 @ T34
  T05 = T01 @ T12 @ T23 @ T34 @ T45 
  T06 = T01 @ T12 @ T23 @ T34 @ T45 @ T56
  T07 = T01 @ T12 @ T23 @ T34 @ T45 @ T56 @ T67
  T0E = T01 @ T12 @ T23 @ T34 @ T45 @ T56 @ T67 @ T7E

  x_pos = [T01[0,-1], T02[0,-1], T03[0,-1], T04[0,-1], T05[0,-1], T06[0,-1], T07[0,-1], T0E[0,-1]]
  y_pos = [T01[1,-1], T02[1,-1], T03[1,-1], T04[1,-1], T05[1,-1], T06[1,-1], T07[1,-1], T0E[1,-1]]
  z_pos = [T01[2,-1], T02[2,-1], T03[2,-1], T04[2,-1], T05[2,-1], T06[2,-1], T07[2,-1], T0E[2,-1]]

  fig = go.Figure()
  fig.add_scatter3d(
      x=np.round(x_pos,2),
      y=np.round(y_pos,2),
      z=z_pos,
      line=dict( color='darkblue', width=15 ),
      hoverinfo="text",
      hovertext=[ f"joint {idx}: {q}" 
          for idx,q in 
            enumerate(np.round(np.rad2deg([ 0, q1, q2, q3, q4, q5, q6, q7 ]),0)) ],
      marker=dict(
          size=10,
          color=[ np.linalg.norm([x,y,z]) for x,y,z in zip(x_pos, y_pos, z_pos) ],
          colorscale='Viridis',
      )
  )
  fig.layout=dict(
      width=1000,
      height=700,
      scene = dict( 
          camera=dict( eye={ 'x':-1.25, 'y':-1.25, 'z':2 } ),
          aspectratio={ 'x':1.25, 'y':1.25, 'z':1 },
          xaxis = dict( nticks=8, ),
          yaxis = dict( nticks=8 ),
          zaxis = dict( nticks=8 ),
          xaxis_title='Robot x-axis',
          yaxis_title='Robot y-axis',
          zaxis_title='Robot z-axis'),
      title=f"Robot in joint Configuration: {np.round(np.rad2deg(q_parms),0)} degrees",
      colorscale=dict(diverging="thermal")
  )
  pio.show(fig)

# plot_robot(np.deg2rad([0,90,45,-90,0,90,0]))

In [17]:
def plot_robots(rob_cnfs, traj_x, traj_y, traj_z, traj_fun=(lambda x, y: 0.5 + 0.5*np.sin(3*x*y))):
  """
  rob_cnfs: list of robot configurations and plots each conf
  
  traj_fun: function defined in terms of x and y and plots the 3D function 
  representing the desired trajectory
  """

  fig = go.Figure()

  fig.add_scatter3d(
        x=traj_x,
        y=traj_y,
        z=traj_z,
        hoverinfo='none',
        marker=dict( size=0.1 ),
        name = "desired trajectory"
  )

  for i,q_params in enumerate(rob_cnfs):
    q1, q2, q3, q4, q5, q6, q7 = q_params

    T01 = np.eye(4)
    T12 = Rz(q1) @ Tz(a1)           # Joint 1 to 2
    T23 = Rx(q2)                    # Joint 2 to 3
    T34 = Rz(q3) @ Tz(a2)           # Joint 3 to 4
    T45 = Rx(q4)                    # Joint 4 to 5
    T56 = Rz(q5) @ Tz(a3)           # Joint 5 to 6
    T67 = Rx(q6)                    # Joint 6 to 7
    T7E = Rz(q7) @ Tz(a4)           # Joint 7 to E
    
    T02 = T01 @ T12
    T03 = T01 @ T12 @ T23
    T04 = T01 @ T12 @ T23 @ T34
    T05 = T01 @ T12 @ T23 @ T34 @ T45 
    T06 = T01 @ T12 @ T23 @ T34 @ T45 @ T56
    T07 = T01 @ T12 @ T23 @ T34 @ T45 @ T56 @ T67
    T0E = T01 @ T12 @ T23 @ T34 @ T45 @ T56 @ T67 @ T7E

    x_pos = [T01[0,-1], T02[0,-1], T03[0,-1], T04[0,-1], T05[0,-1], T06[0,-1], T07[0,-1], T0E[0,-1]]
    y_pos = [T01[1,-1], T02[1,-1], T03[1,-1], T04[1,-1], T05[1,-1], T06[1,-1], T07[1,-1], T0E[1,-1]]
    z_pos = [T01[2,-1], T02[2,-1], T03[2,-1], T04[2,-1], T05[2,-1], T06[2,-1], T07[2,-1], T0E[2,-1]]

    # Setting opacity
    if (i != 0 and i != len(rob_cnfs)-1): 
      opc=0.3
    else: 
      opc=1
    
    # Width
    wd=10

    fig.add_scatter3d(
        x=np.round(x_pos,2),
        y=np.round(y_pos,2),
        z=z_pos,
        line=dict( color='darkblue', width=wd ),
        hoverinfo="text",
        hovertext=[ f"joint {idx}: {q}" 
            for idx,q in 
              enumerate(np.round(np.rad2deg([ 0, q1, q2, q3, q4, q5, q6, q7 ]),0)) ],
        marker=dict(
            size=wd/2,
            color=["black", "orange", "yellow", "pink", "blue", "goldenrod", "green", "red"],
            # colorscale='Viridis',
        ),
        opacity=opc,
        showlegend=False,
        name = f"conf {i}"
    )


  fig.layout=dict(
    width=1000,
    height=700,
    scene = dict( 
        camera=dict( eye={ 'x':-1.25, 'y':-1.25, 'z':2 } ),
        aspectratio={ 'x':1.25, 'y':1.25, 'z':1 },
        xaxis = dict( nticks=8, range=[np.min(traj_x)-0.5, np.max(traj_x)+0.5] ),
        yaxis = dict( nticks=8, range=[np.min(traj_y)-0.5, np.max(traj_y)+0.5] ),
        zaxis = dict( nticks=8, range=[-0.05, np.max(traj_z)+0.4] ),
        xaxis_title='Robot x-axis',
        yaxis_title='Robot y-axis',
        zaxis_title='Robot z-axis'),
    title=f"Robot plot in different configurations",
    colorscale=dict(diverging="thermal")
  )
  pio.show(fig)

# plot_robots([np.deg2rad([0,90,0,-90,0,90,0]), np.deg2rad([0,0,0,0,0,0,0]), np.deg2rad([0,-90,0,90,0,-90,0])])

### <b><i>Redundancy Analysis</i></b>

In [18]:
def get_cnfs(method_fun, q0=np.deg2rad([0,30,0,-20,0,45,0]), kwargs=dict()):
  """Returns all the joint configurations for the robot at different points on 
    the required trajectory for a specific method"""

  x = np.hstack([
          [ 0.4 for _ in range(5) ],
          [ 0.2 for _ in range(5)], 
          np.linspace(0.2, 0.4, 5), 
          np.linspace(0.2, 0.4, 5) ])
  
  y = np.hstack([
          np.linspace(0.2, 0.4, 5), 
          np.linspace(0.2, 0.4,5), 
          [ 0.4 for _ in range(5) ], 
          [ 0.2 for _ in range(5)]]) 
  
  z = [ 0.9 for _ in x ]

  rob_cnfs = []   # will contain the result of each inverse kinematics

  start_time = time.time()

  for (i, j, k) in zip (x, y, z):
    pos = [i, j, k]

    q = method_fun(q0, pos, **kwargs)

    rob_cnfs.append(q)
    # q0 = q     # Sets the new initial joint configurations to the previous

  end_time = time.time()

  print(f"\n{np.round(end_time-start_time, 1)} seconds : Total time using {method_fun.__name__} \n")
  if kwargs: print(f"\nParameters used: {kwargs}")

  plot_robots(rob_cnfs, traj_x=x, traj_y=y, traj_z=z)

##### <i>Simple Pseudo Inverse</i>

In [19]:
def simple_pseudo(q0, p_goal, time_step=0.01, max_iteration=3000, accuracy=0.001):

  assert np.linalg.norm(p_goal) <= 0.85*np.sum([a1, a2, a3, a4]), "Robot Length constraint violated"

  # Setting initial variables
  q_n0 = q0
  p = FK(q_n0)[:3,-1]

  t_dot = p_goal - p
  e = np.linalg.norm(t_dot)

  Tt = np.block([ np.eye(3), np.zeros((3,3)) ])
  q_n1 = q_n0
  δt = time_step
  i=0

  start_time = time.time()

  while True:
    if (e < accuracy): 
      print(f"Accuracy of {accuracy} reached")
      break

    p = FK(q_n0)[:3,-1]
    t_dot = p_goal - p
    e = np.linalg.norm(t_dot)

    J_inv = np.linalg.pinv( (Tt @ jacobian(q_n0)) )  
    q_dot = J_inv @ t_dot
    q_n1 = q_n0 + (δt * q_dot)  

    q_n0 = q_n1

    i+=1
    if (i > max_iteration):
      print("No convergence")
      break
    
  end_time = time.time()
  print(f"Total time taken {np.round(end_time - start_time, 4)} seconds\n")

  return np.mod(q_n1, 2*np.pi)

##### <i>Weighted Pseudo Inverse</i> 

In [20]:
def weighted_pseudo(q0, p_goal, weights=[10,30,25,20,90,42,57], time_step=0.01, max_iteration=3000, accuracy=0.001):

  assert np.linalg.norm(p_goal) < 0.85*np.sum([a1, a2, a3, a4]), "Robot Length constraint violated"

  q_n0 = q0
  p = FK(q_n0)[:3,-1]
  t1_dot = p_goal - p
  e = np.linalg.norm(t1_dot)

  Tt = np.eye(6)[:3]
  v1 = np.zeros(3)
  q_n1 = q_n0
  q_dot = np.zeros(len(q0))
  δt = time_step
  i=0

  start_time = time.time()
  while True:
    if (e < accuracy): 
      break
    
    fk = FK(q_n0)

    p = fk[:3,-1]

    t1_dot = p_goal - p

    e = np.linalg.norm(t1_dot)

    J1 = jacobian(q_n0)[:3]

    w_inv = np.linalg.inv(np.diag(weights)) 
    j1_hash = w_inv @ J1.T @ np.linalg.inv( J1 @ w_inv @ J1.T )

    q_dot = j1_hash @ t1_dot

    q_n1 = q_n0 + (δt * q_dot)  

    q_n0 = q_n1

    i+=1
    if (i > max_iteration):
      print("No convergence")
      break
    
  end_time = time.time()
  print(f"to {np.round(p_goal,2)} :: {np.round(end_time - start_time, 4)} seconds\n")

  return np.mod(q_n1, 2*np.pi)

In [21]:
get_cnfs(method_fun=weighted_pseudo, q0=np.deg2rad([0,30,0,-20,0,45,0]))

to [ 1.13  0.95 -0.88 -1.56  0.07  1.17  0.  ] :: 7.667 seconds

to [ 1.22  0.9  -0.97 -1.51  0.08  1.14  0.  ] :: 7.2964 seconds

to [ 1.32  0.86 -1.06 -1.44  0.08  1.1   0.  ] :: 7.5626 seconds

to [ 1.44  0.83 -1.16 -1.36  0.09  1.05  0.  ] :: 7.7338 seconds

to [ 1.58  0.81 -1.26 -1.26  0.09  1.    0.  ] :: 7.4591 seconds

to [ 0.45  0.59 -0.6  -1.87  0.05  1.17  0.  ] :: 7.2655 seconds

to [ 0.42  0.46 -0.64 -1.82  0.05  1.14  0.  ] :: 7.2747 seconds

to [ 0.37  0.32 -0.66 -1.75  0.05  1.1   0.  ] :: 7.4143 seconds

to [ 0.29  0.18 -0.66 -1.67  0.05  1.07  0.  ] :: 7.2412 seconds

to [ 0.22  0.04 -0.64 -1.58  0.05  1.04  0.  ] :: 7.5885 seconds

to [ 0.22  0.04 -0.64 -1.58  0.05  1.04  0.  ] :: 7.5095 seconds

to [ 0.32 -0.02 -0.88 -1.53  0.06  0.98  0.  ] :: 7.4292 seconds

to [ 0.66 -0.09 -1.38 -1.46  0.1   0.66  0.  ] :: 7.3124 seconds

to [ 1.45  0.64 -1.36 -1.37  0.1   0.94  0.  ] :: 7.7651 seconds

to [ 1.58  0.81 -1.26 -1.26  0.09  1.    0.  ] :: 7.6993 seconds

to [ 0.45  

##### <i>Damped Least Square</i> 

In [22]:
def damped_least_squares(q0, p_goal, lmbda = sqrt(0.01), time_step=0.01, max_iteration=3000, accuracy=0.001):

  assert np.linalg.norm(p_goal[:3]) <= 0.85*np.sum([a1, a2, a3, a4]), "Robot Length constraint violated"

  q_n0 = q0
  p = FK(q_n0)[:3,-1]
  t_dot = p_goal[:3] - p
  e = np.linalg.norm(t_dot)

  Tt = np.block([ np.eye(3), np.zeros((3,3)) ])
  q_n1 = q_n0
  δt = time_step
  i=0

  start_time = time.time()
  while True:
    if (e < accuracy): 
      break
    
    fk = FK(q_n0)

    rx, ry, rz = euler_angles(fk[:3,:3])

    p = np.hstack([fk[:3,-1], [rx, ry, rz] ]) # current position and orientation

    t_dot = p_goal[:3] - p[:3]

    e = np.linalg.norm(t_dot)

    Jt = np.dot(Tt, jacobian(q_n0))

    j_star = (Jt @ Jt.T)  + (lmbda**2 * np.eye(3))

    J_dls = Jt.T @ np.linalg.inv( j_star )

    q_dot = np.linalg.multi_dot([J_dls, t_dot])

    q_n1 = q_n0 + (δt * q_dot)  

    q_n0 = q_n1

    i+=1
    if (i > max_iteration):
      print("No convergence")
      break
    
  end_time = time.time()
  print(f"to {np.round(p_goal,2)} :: time taken {np.round(end_time - start_time, 4)} seconds\n")

  return np.mod(q_n1, 2*np.pi)

In [23]:
get_cnfs(method_fun=damped_least_squares, q0=np.deg2rad([0,0,0,0,0,0,0]))

to [0.4 0.2 0.9] :: time taken 7.4408 seconds

to [0.4  0.25 0.9 ] :: time taken 7.3123 seconds

to [0.4 0.3 0.9] :: time taken 7.5034 seconds

to [0.4  0.35 0.9 ] :: time taken 7.5686 seconds

to [0.4 0.4 0.9] :: time taken 7.6868 seconds

to [0.2 0.2 0.9] :: time taken 7.1947 seconds

to [0.2  0.25 0.9 ] :: time taken 7.4781 seconds

to [0.2 0.3 0.9] :: time taken 7.3795 seconds

to [0.2  0.35 0.9 ] :: time taken 7.5073 seconds

to [0.2 0.4 0.9] :: time taken 7.1419 seconds

to [0.2 0.4 0.9] :: time taken 7.5232 seconds

to [0.25 0.4  0.9 ] :: time taken 7.4694 seconds

to [0.3 0.4 0.9] :: time taken 7.1723 seconds

to [0.35 0.4  0.9 ] :: time taken 7.5971 seconds

to [0.4 0.4 0.9] :: time taken 7.9788 seconds

to [0.2 0.2 0.9] :: time taken 7.8879 seconds

to [0.25 0.2  0.9 ] :: time taken 8.1772 seconds

to [0.3 0.2 0.9] :: time taken 7.5159 seconds

to [0.35 0.2  0.9 ] :: time taken 7.5887 seconds

to [0.4 0.2 0.9] :: time taken 7.4943 seconds


150.7 seconds : Total time using da

##### <i>Null space Method</i> 

In [41]:
def null_space_method(q0, p_goal, weights=[1,3,1,2,9,4,5], time_step=0.01, max_iteration=3000, accuracy=0.01):

  assert np.linalg.norm(p_goal[:3]) <= 0.85*np.sum([a1, a2, a3, a4]), "Robot Length constraint violated"

  q_n0 = q0
  p = FK(q_n0)[:3,-1]
  t_dot = p_goal[:3] - p
  H1 = [0, 0, 0, 0, 0, 0, 0]
  e = np.linalg.norm(t_dot)

  Tt = np.block([ np.eye(3), np.zeros((3,3)) ])
  q_n1 = q_n0
  δt = time_step
  q_dot_0 =  [ 4, 9, 1, 7, 9, 5, 1] # [ 0.4, 0.9, 0.1, 0.7, 0.9, 0.5, 0.22]
  i=0

  start_time = time.time()
  while True:
    if (e < accuracy): 
      break
    
    fk = FK(q_n0)

    rx, ry, rz = euler_angles(fk[:3,:3])

    p = np.hstack([fk[:3,-1], [rx, ry, rz] ]) # current position and orientation

    t_dot = p_goal[:3] - p[:3]

    e = np.linalg.norm(t_dot)

    w_inv = np.linalg.inv(np.diag(weights)) 

    Jt = np.dot(Tt, jacobian(q_n0))

    j_hash = w_inv @ Jt.T @ np.linalg.inv( Jt @ w_inv @ Jt.T )


    q_dot = (j_hash @ t_dot) + (np.eye(7) - (j_hash @ Jt))@q_dot_0

    q_n1 = q_n0 + (δt * q_dot)  

    # H2 = np.sqrt(np.linalg.det(Jt@Jt.T))

    # dH = H2 - H1

    # dq = q_n1 - q_n0

    # grad_Hq = np.divide(dH, dq)

    # H2 = []
    # for i,q in enumerate(q_n1):
    #   q_i = q_n1          # np.zeros(7)
    #   q_i[i]=q-q_n0[i]
    #   Jt_i = np.dot(Tt, jacobian(q_i))

    #   H2.append(np.sqrt(np.linalg.det(Jt_i@Jt_i.T)))

    # grad_Hq = np.asarray(H2) - np.asarray(H1)

    # q_dot_0 = grad_Hq.T

    q_n0 = q_n1
    # H1 = H2

    i+=1
    if (i > max_iteration):
      print("No convergence")
      break
    
  end_time = time.time()
  print(f"to {np.round(p_goal,2)} :: time taken {np.round(end_time - start_time, 4)} seconds\n")

  return np.mod(q_n1, 2*np.pi)

In [44]:
get_cnfs(method_fun=null_space_method, q0=np.deg2rad([10,-10,0,20,0,-60,0]))

to [0.4 0.2 0.9] :: time taken 5.1094 seconds

to [0.4  0.25 0.9 ] :: time taken 4.9411 seconds

to [0.4 0.3 0.9] :: time taken 4.9785 seconds

to [0.4  0.35 0.9 ] :: time taken 4.457 seconds

to [0.4 0.4 0.9] :: time taken 4.6122 seconds

to [0.2 0.2 0.9] :: time taken 3.8936 seconds

to [0.2  0.25 0.9 ] :: time taken 3.9871 seconds

to [0.2 0.3 0.9] :: time taken 4.6807 seconds

to [0.2  0.35 0.9 ] :: time taken 4.7722 seconds

to [0.2 0.4 0.9] :: time taken 4.4883 seconds

to [0.2 0.4 0.9] :: time taken 4.4019 seconds

to [0.25 0.4  0.9 ] :: time taken 4.6433 seconds

to [0.3 0.4 0.9] :: time taken 4.4706 seconds

to [0.35 0.4  0.9 ] :: time taken 4.8751 seconds

to [0.4 0.4 0.9] :: time taken 4.5177 seconds

to [0.2 0.2 0.9] :: time taken 4.3473 seconds

to [0.25 0.2  0.9 ] :: time taken 4.6315 seconds

to [0.3 0.2 0.9] :: time taken 4.6757 seconds

to [0.35 0.2  0.9 ] :: time taken 4.7994 seconds

to [0.4 0.2 0.9] :: time taken 4.4672 seconds


91.8 seconds : Total time using null

##### <b><i>Taking into account Task Priority</i></b> 

In [31]:
def get_cnfs_priority(method_fun, q0=np.deg2rad([0,30,0,-20,0,45,0]), kwargs=dict()):
  """Returns all the joint configurations for the robot at different points on 
    the required trajectory for a specific method"""

  x = np.hstack([
          [ 0.4 for _ in range(5) ],
          [ 0.2 for _ in range(5)], 
          np.linspace(0.2, 0.4, 5), 
          np.linspace(0.2, 0.4, 5) ])
  
  y = np.hstack([
          np.linspace(0.2, 0.4, 5), 
          np.linspace(0.2, 0.4,5), 
          [ 0.4 for _ in range(5) ], 
          [ 0.2 for _ in range(5)]]) 
  
  z = [ 0.9 for _ in x ]

  rx = [ np.pi/2 for _ in x ]
  ry = [ 0 for _ in x ]
  rz = [ 0 for _ in x ]

  rob_cnfs = []   # will contain the result of each inverse kinematics

  start_time = time.time()

  for (i, j, k, phi_x, phi_y, phi_z) in zip (x, y, z, rx, ry, rz):
    pos = [i, j, k, phi_x, phi_y, phi_z]

    q = method_fun(q0, pos, **kwargs)

    rob_cnfs.append(q)
    # q0 = q     # Sets the new initial joint configurations to the previous

  end_time = time.time()

  print(f"\n{np.round(end_time-start_time, 1)} seconds : Total time using {method_fun.__name__} \n")
  if kwargs: print(f"\nParameters used: {kwargs}")

  plot_robots(rob_cnfs, traj_x=x, traj_y=y, traj_z=z)

In [32]:
# Using Damped Least Square
def dls_priority(q0, p_goal, lmbda = sqrt(0.01), time_step=0.1, max_iteration=3000, accuracy=0.001):

  assert np.linalg.norm(p_goal[:3]) < 0.85*np.sum([a1, a2, a3, a4]), "Robot Length constraint violated"

  p = FK(q0)[:3,-1]
  t_dot = p_goal[:3] - p
  e = np.linalg.norm(t_dot)
  q_n0 = q0
  q_n1 = q0

  i=0

  start_time = time.time()
  while True:
    if (e < accuracy): 
      break
    
    fk = FK(q_n0)

    rx, ry, rz = euler_angles(fk[:3,:3])

    p = np.hstack([fk[:3,-1], [rx, ry, rz] ]) # current position and orientation

    t_dot1 = p_goal[:3] - p[:3]
    t_dot2 = p_goal[3:5] - p[3:5]

    e = np.linalg.norm(t_dot1)

    Jt1 = jacobian(q_n0)[:3]
    Jt2 = jacobian(q_n0)[3:5]

    Jt1_dls = Jt1.T @ np.linalg.inv( (Jt1 @ Jt1.T)  + (lmbda**2 * np.eye(3)) )
    p1 = np.eye(7) - (Jt1_dls @ Jt1)
    
    Jt2_term = Jt2 @ p1
    Jt2_dls = Jt2_term.T @ np.linalg.inv( (Jt2_term @ Jt2_term.T)  + (lmbda**2 * np.eye(2)) )

    v1 = Jt2_dls @ (t_dot2 - Jt2 @ Jt1_dls @ t_dot1)
    q_dot = (Jt1_dls @ t_dot1) + p1 @ v1

    q_n1 = q_n0 + (time_step * q_dot)  

    q_n0 = q_n1

    i+=1
    if (i > max_iteration):
      print("No convergence")
      break
    
  end_time = time.time()
  print(f"to {np.round(p_goal,2)} :: time taken {np.round(end_time - start_time, 4)} seconds\n")

  return np.mod(q_n1, 2*np.pi)

In [33]:
get_cnfs_priority(method_fun=dls_priority, q0=np.deg2rad([0,0,0,0,0,0,0]), kwargs=dict(accuracy=0.001))

to [0.4  0.2  0.9  1.57 0.   0.  ] :: time taken 2.317 seconds

to [0.4  0.25 0.9  1.57 0.   0.  ] :: time taken 10.0191 seconds

to [0.4  0.3  0.9  1.57 0.   0.  ] :: time taken 2.4136 seconds

to [0.4  0.35 0.9  1.57 0.   0.  ] :: time taken 2.2939 seconds

to [0.4  0.4  0.9  1.57 0.   0.  ] :: time taken 25.9507 seconds

to [0.2  0.2  0.9  1.57 0.   0.  ] :: time taken 1.5042 seconds

to [0.2  0.25 0.9  1.57 0.   0.  ] :: time taken 3.8181 seconds

to [0.2  0.3  0.9  1.57 0.   0.  ] :: time taken 2.9514 seconds

to [0.2  0.35 0.9  1.57 0.   0.  ] :: time taken 3.6462 seconds

to [0.2  0.4  0.9  1.57 0.   0.  ] :: time taken 5.3422 seconds

to [0.2  0.4  0.9  1.57 0.   0.  ] :: time taken 5.2153 seconds

to [0.25 0.4  0.9  1.57 0.   0.  ] :: time taken 5.1327 seconds

to [0.3  0.4  0.9  1.57 0.   0.  ] :: time taken 1.6475 seconds

to [0.35 0.4  0.9  1.57 0.   0.  ] :: time taken 1.9317 seconds

to [0.4  0.4  0.9  1.57 0.   0.  ] :: time taken 24.9656 seconds

to [0.2  0.2  0.9  1.57

In [ ]:
# Using Weighted Psuedo Inverse
def wpi_priority(q0, p_goal, weights=[1,3,1,2,9,4,5], time_step=0.01, max_iteration=3000, accuracy=0.01):

  assert np.linalg.norm(p_goal[:3]) < 0.7*np.sum([a1, a2, a3, a4]), "Robot Length constraint violated"

  q_n0 = q0
  p = FK(q_n0)[:3,-1]
  t1_dot = p_goal[:3] - p
  e = np.linalg.norm(t1_dot)

  Tt = np.eye(6)[:3]
  v1 = np.zeros(3)
  q_n1 = q_n0
  q_dot = np.zeros(len(q0))
  δt = time_step
  i=0

  start_time = time.time()
  while True:
    if (e < accuracy): 
      break
    
    fk = FK(q_n0)

    rx, ry, rz = euler_angles(fk[:3,:3])

    p = np.hstack([fk[:3,-1], [rx, ry, rz] ]) # current position and orientation

    t1_dot = p_goal[:3] - p[:3]
    t2_dot = p_goal[3:5] - p[3:5]

    e = np.linalg.norm(t1_dot)

    J1 = jacobian(q_n0)[:3]
    J2 = jacobian(q_n0)[3:5]

    w_inv = np.linalg.inv(np.diag(weights)) 
    j1_hash = w_inv @ J1.T @ np.linalg.inv( J1 @ w_inv @ J1.T )

    t2_dot = J2 @ q_dot

    p1 = (np.eye(7) - (j1_hash @ J1))

    j2_res = J2 @ p1
    j2_hash = w_inv @ j2_res.T @ np.linalg.inv( j2_res @ w_inv @ j2_res.T )

    q_dot = (j1_hash @ t1_dot) + (p1 @ j2_hash) @ (t2_dot - J2 @ j1_hash @ t1_dot) 

    q_n1 = q_n0 + (δt * q_dot)  

    q_n0 = q_n1

    i+=1
    if (i > max_iteration):
      print("No convergence")
      break
    
  end_time = time.time()
  print(f"to {np.round(q_n0,1)} :: {np.round(end_time - start_time, 4)} seconds\n")

  return np.mod(q_n1, 2*np.pi)